In [1]:
!pip install -q \
  langchain==0.1.16 \
  langchain-community==0.0.38 \
  langchain-openai==0.1.6 \
  tavily-python \
  requests==2.32.5

In [2]:
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
from langchain.memory import ConversationBufferMemory

from google.colab import userdata
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")

Importing openai and tavily api keys

In [3]:
llm = ChatOpenAI(model="gpt-4o", temperature=0.3)
search_tool = TavilySearchResults(k=3)

def get_weather(city):
  return requests.get(f"https://wttr.in/{city}?format=3").text

weather_tool = Tool(
    name="WeatherTool",
    func=get_weather,
    description="Getting weather"
)
tools = [search_tool,weather_tool]



We are using gpt 4o mini due to it's low cost and high speed. And we are using tavily search with top 3 search results. Weather tool to help the model get the accurate weather details for a location.

In [4]:
system_prompt = """
You are an AI travel planning agent.

MANDATORY RULES (failure = regenerate):
1. Every itinerary MUST include a dedicated section called "Accommodation Options".
2. This section MUST list 2–3 REAL hotel names.
3. Each hotel MUST include:
   - Hotel name
   - Area/locality
   - Price per night in INR
   - Booking source (Booking.com, MakeMyTrip, Goibibo, etc.)
4. Hotel prices MUST scale with the total trip budget.
5. You MUST use web search to find hotels. Do not invent names.
6. If hotel names are missing, the answer is INVALID.

Hotel budget tiers:
- Total budget < ₹30,000 → ₹1,500–₹3,000/night
- ₹30,000–₹70,000 → ₹3,000–₹6,000/night
- ₹70,000–₹1,20,000 → ₹6,000–₹10,000/night
- > ₹1,20,000 → ₹10,000–₹18,000/night

OUTPUT FORMAT (STRICT):

Accommodation Options:
- Hotel Name – Area – ₹price/night (source)

Then provide day-wise itinerary and cost breakdown.
"""



Initial prompt to the LLM.

In [5]:
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    memory=memory,
    verbose=True
)

/usr/local/lib/python3.12/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [6]:
INTEREST_KEYWORDS = {
    "Adventure": [
        "adventure", "trek", "hiking", "rafting", "sports",
        "thrill", "explore", "zipline"
    ],
    "Food": [
        "food", "eat", "cuisine", "restaurant", "street food",
        "local food", "dishes"
    ],
    "Nature": [
        "nature", "mountains", "beach", "forest", "relax",
        "scenery", "peaceful"
    ],
    "History": [
        "history", "heritage", "monuments", "museum",
        "culture", "ancient"
    ]
}


In [7]:
def extract_interests_from_text(user_text):
    user_text = user_text.lower()
    detected_interests = set()

    for interest, keywords in INTEREST_KEYWORDS.items():
        for keyword in keywords:
            if keyword in user_text:
                detected_interests.add(interest)
                break

    if not detected_interests:
        detected_interests.add("General")

    return list(detected_interests)


In [8]:
def initialize_state(destination, days, budget, user_text):
    interests = extract_interests_from_text(user_text)

    return {
        "destination": destination,
        "days": days,
        "budget": budget,
        "interests": interests,
        "user_text": user_text
    }


In [9]:
def analyze_constraints(state):
    return {
        "low_budget": state["budget"] < 30000,
        "medium_budget": 30000 <= state["budget"] < 70000,
        "high_budget": state["budget"] >= 70000,
        "interests": state["interests"]
    }

def generate_activity_pool(constraints):
    activity_map = {
        "Adventure": ["Trekking", "Water sports", "Adventure activity"],
        "Food": ["Street food tour", "Local restaurant visit"],
        "History": ["Museum visit", "Heritage walk"],
        "Nature": ["Nature walk", "Scenic relaxation"],
        "General": ["City exploration"]
    }

    activities = []
    for interest in constraints["interests"]:
        activities.extend(activity_map.get(interest, []))

    return activities


def optimize_activities_by_budget(activities, constraints):
    if constraints["low_budget"]:
        return activities[:2]
    elif constraints["medium_budget"]:
        return activities[:4]
    else:
        return activities

def build_itinerary(state, activities):
    itinerary = []
    index = 0

    for day in range(1, state["days"] + 1):
        daily_activities = activities[index:index + 2]

        if not daily_activities:
            daily_activities = ["Free exploration"]

        plan = (
            f"Day {day} in {state['destination']}: "
            + ", ".join(daily_activities)
        )

        itinerary.append(plan)
        index += 2

    return itinerary


In [10]:
def travel_planner_agent(destination, days, budget, user_text):
    prompt = f"""
    User preferences: {user_text}
    Destination: {destination}
    Days: {days}
    Budget: {budget}

    Tasks:
    1. Infer user interests but don't give any output just use it for the iterinary.
    2. Search for hotels in {destination} using Tavily and airfare to and from {destination} using Tavily if origin provided by user.
    3. For interested activities search in platforms like tripadvisor using Tavily.
    4. Create a day-wise itinerary using the recommended hotels.
    5. Don't use bold or italics.

    IMPORTANT: You MUST use the Tavily search tool.
    """

    response = agent.run(prompt)
    return response


In [11]:
output = travel_planner_agent(
    destination="Goa",
    days=3,
    budget=50000,
    user_text="I want to try local food and do adventurous things but also relax near nature. Im going from Chennai."
)

print(output)


/usr/local/lib/python3.12/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}